In [44]:
%%file encoding.lp
%%clingo 0 instances/env2.lp -
%%file encoding.lp
%%clingo 0 instances/env4.lp -



#const n=4.
time(1..n).

%_________________Encode Tracks____________________

%straight tracks
track(1025, (1, 3)). track(32800, (0, 2)). track(4608, (3, 2)). track(16386, (2, 1)). track(72, (0, 1)). track(2064, (0, 3)).

%simple switch
track(3089, (3, 1)). track(3089, (3, 0)). track(37408, (3, 2)). track(37408, (0, 2)). track(17411, (2, 3)). track(17411, (1, 3)).
track(32872, (0, 1)). track(32872, (0, 2)). track(49186, (0, 2)). track(49186, (2, 1)). track(1097, (1, 3)). track(1097, (0, 1)).
track(34864, (3, 0)). track(34864, (0, 2)). track(5633, (3, 1)). track(5633, (3, 2)).

%diamond crossing
track(33825, (0, 2)). track(33825, (1, 3)).

%diamond plus crossing
track(38433, (0, 2)). track(38433, (1, 3)). track(38433, (3, 2)). track(50211, (0, 2)). track(50211, (1, 3)). track(50211, (2, 1)).
track(33897, (0, 2)). track(33897, (1, 3)). track(33897, (0, 1)). track(35889, (0, 2)). track(35889, (1, 3)). track(35889, (0, 3)).
track(38505, (0, 2)). track(38505, (1, 3)). track(38505, (3, 2)). track(38505, (0, 1)).
track(52275, (0, 2)). track(52275, (1, 3)). track(52275, (2, 1)). track(52275, (0, 3)).

%double curve crossing
track(20994, (1, 2)). track(20994, (2, 3)). track(16458, (0, 1)). track(16458, (1, 2)). 
track(2136, (0, 1)). track(2136, (3, 0)). track(6672, (2, 3)). track(6672, (3, 0)).

%track end
track(8192, (2, end)). track(4, (1, end)). track(128, (0, end)). track(256, (3, end)).

%_________________Generate__________________________

%single slip switch 

track(38505, (0, 2)).
track(38505, (1, 3)).
track(38505, (2, 3)).
track(38505, (0, 1)).

track(52275, (0, 2)).
track(52275, (1, 3)).
track(52275, (0, 3)).
track(52275, (2, 1)).

%symmetrical switch

track(20994, (2, 1)).
track(20994, (2, 3)).

track(16458, (0, 1)).
track(16458, (1, 2)).

track(2136, (0, 1)).
track(2136, (0, 3)).

track(6672, (3, 0)).
track(6672, (3, 2)).

track(N, (X, Y)) :- track(N, (Y, X)).

moveDir(0, (0, 1)). %go north 
moveDir(1, (1, 0)). %go east
moveDir(2, (0, -1)). %go south
moveDir(3, (-1, 0)). %go west

revDir(0, 2). revDir(2, 0). revDir(1, 3). revDir(3, 1).

dir2int(n, 0).
dir2int(e, 1).
dir2int(s, 2).
dir2int(w, 3).

move(Type, DirL, move_left):- track(Type, (DirIn, DirL)), track(Type, (DirIn, DirR)), DirL = DirIn + 1, DirL != DirR.
move(Type, 0, move_left):- track(Type, (3, 0)), track(Type, (3, DirR)), DirR > 0.
    
move(Type, DirR, move_right):- track(Type, (DirIn, DirL)), track(Type, (DirIn, DirR)), DirR = DirIn - 1, DirL != DirR.
move(Type, 3, move_right):- track(Type, (0, 3)), track(Type, (0, DirL)), DirL < 3.
    
move(Type, DirOut, move_forward) :- track(Type, (DirIn, DirOut)), |DirOut - DirIn| = 2.
move(Type, DirOut, move_forward) :- track(Type, (DirIn, DirOut)),
    not move(Type, DirOut, move_left),
    not move(Type, DirOut, move_right).    
    
agent(A) :- start(agent(A), _, _).
action_type(move_forward). action_type(wait). action_type(move_left). action_type(move_right).
    
{action(agent(A), Action, Timestamp)} :- start(agent(A),(Y,X),dir(D)), action_type(Action), time(Timestamp).
    
%action(agent(A), wait, 0) :- agent(A).
%one action for each agent and each timestamp
:- time(T), agent(A), not 1 {action(agent(A), _, T)} 1.
     
current_state(agent(A), (Y,X), 0, Dir) :- start(agent(A),(Y,X),dir(D)), dir2int(D, Dir).  
    
current_state(agent(A), (Y + DY, X+DX), T + 1, DirOut2) :- current_state(agent(A), (Y,X), T, DirOut),
    action(agent(A), Action, T + 1), cell((Y,X), Type), cell((Y + DY, X+DX), Type2),
    track(Type, (_, DirOut)), track(Type2, (DirIn,DirOut2)), move(Type, DirOut, Action),
    revDir(DirIn, DirOut), moveDir(DirOut, (DY,DX)).

current_state(agent(A), (Y,X), T + 1, Dir) :- current_state(agent(A), (Y,X), T, Dir), action(agent(A), wait, T + 1).
    
:- action(agent(A), _, T), not current_state(agent(A), _, T, _).

%vertex conflict
:-current_state(agent(A1), (Y,X), T, _), current_state(agent(A2), (Y,X), T, _), A1 < A2.

%also include edge conflict later 
    
%agents have to reach destination
:- end(agent(A), (Y,X)), not current_state(agent(A), (Y,X), _, _).

    
%#show action/3.
#show current_state/4.
%#show move/3.
%#show track/2.

Overwriting encoding.lp


In [36]:
s ="move(32872,1,move_left) move(38505,1,move_left) move(52275,2,move_left) move(20994,3,move_left) move(2136,1,move_left) move(16458,2,move_left) move(38505,3,move_left) move(37408,3,move_left) move(3089,0,move_left) move(6672,0,move_left) move(52275,0,move_left) move(49186,1,move_right) move(5633,2,move_right) move(52275,1,move_right) move(20994,1,move_right) move(6672,2,move_right) move(16458,0,move_right) move(38505,0,move_right) move(38505,2,move_right) move(1097,0,move_right) move(17411,2,move_right) move(52275,3,move_right) move(2136,3,move_right) move(34864,3,move_right) move(1025,3,move_forward) move(32800,2,move_forward) move(3089,1,move_forward) move(37408,2,move_forward) move(17411,3,move_forward) move(32872,2,move_forward) move(49186,2,move_forward) move(1097,3,move_forward) move(34864,2,move_forward) move(5633,1,move_forward) move(33825,2,move_forward) move(33825,3,move_forward) move(38505,2,move_forward) move(38505,3,move_forward) move(52275,2,move_forward) move(52275,3,move_forward) move(52275,1,move_forward) move(52275,0,move_forward) move(38505,1,move_forward) move(38505,0,move_forward) move(33825,1,move_forward) move(33825,0,move_forward) move(5633,3,move_forward) move(34864,0,move_forward) move(1097,1,move_forward) move(49186,0,move_forward) move(32872,0,move_forward) move(17411,1,move_forward) move(37408,0,move_forward) move(3089,3,move_forward) move(32800,0,move_forward) move(1025,1,move_forward) move(4608,2,move_forward) move(16386,1,move_forward) move(72,1,move_forward) move(2064,3,move_forward) move(16458,1,move_forward) move(6672,3,move_forward) move(2136,0,move_forward) move(20994,2,move_forward) move(2064,0,move_forward) move(72,0,move_forward) move(16386,2,move_forward) move(4608,3,move_forward)"

In [37]:
a = s.split()
a.sort()
print(*a, sep="\n")

move(1025,1,move_forward)
move(1025,3,move_forward)
move(1097,0,move_right)
move(1097,1,move_forward)
move(1097,3,move_forward)
move(16386,1,move_forward)
move(16386,2,move_forward)
move(16458,0,move_right)
move(16458,1,move_forward)
move(16458,2,move_left)
move(17411,1,move_forward)
move(17411,2,move_right)
move(17411,3,move_forward)
move(2064,0,move_forward)
move(2064,3,move_forward)
move(20994,1,move_right)
move(20994,2,move_forward)
move(20994,3,move_left)
move(2136,0,move_forward)
move(2136,1,move_left)
move(2136,3,move_right)
move(3089,0,move_left)
move(3089,1,move_forward)
move(3089,3,move_forward)
move(32800,0,move_forward)
move(32800,2,move_forward)
move(32872,0,move_forward)
move(32872,1,move_left)
move(32872,2,move_forward)
move(33825,0,move_forward)
move(33825,1,move_forward)
move(33825,2,move_forward)
move(33825,3,move_forward)
move(34864,0,move_forward)
move(34864,2,move_forward)
move(34864,3,move_right)
move(37408,0,move_forward)
move(37408,2,move_forward)
move(37408,3,m